In [ ]:
# PS-7 Password Generation
# This notebook now uses the modular pipeline

from password_generator import PasswordGenerator
from pathlib import Path
import config

print('✓ Password generator module imported')
print(f'\nTo generate candidates: python pipeline.py generate')
print(f'Or use PasswordGenerator class below.')

In [ ]:
# Initialize generator
generator = PasswordGenerator(
    Path('pcfg_of_len6_or_more.json'),
    Path('frag_tokens_all.tsv')
)

print(f'Generator loaded with:')
print(f'  Templates: {len(generator.top_templates)}')
print(f'  Words: {len(generator.top_words)}')
print(f'  Digits: {len(generator.top_digits)}')
print(f'  Fragments: {len(generator.top_frags)}')

# Generate some examples
print('\nExample deterministic candidates from template FRAG|DIGITS2:')
for i, (cand, score) in enumerate(generator.generate_deterministic('FRAG|DIGITS2', max_out=10)):
    print(f'{i+1:2}. {cand:20} (score: {score:.2f})')